In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import numpy as np

from torch.optim.optimizer import Optimizer, required
import dill as pickle

from torch.distributions.normal import Normal

from tqdm import tqdm

import pysgmcmc
from scipy.stats import wasserstein_distance
import sys
import heteroskedastic_nns.datasets.load_uci
#from heteroskedastic_nns.parallel_model import ParallelFF
#from heteroskedastic_nns.model import SingleFF
from heteroskedastic_nns.datasets.generate_data import prep_synth_data, prep_uci_data
from heteroskedastic_nns.utils.utils import plot_result, make_heatmap, plot_parallel_model, plot_split_model, num_grad, vec_num_grad, mult_approx_grad, gam_rho_to_alpha_beta, expected_calibration_error_nfe
from torchquad import MonteCarlo, set_up_backend
import math
from tabulate import tabulate
import copy

from matplotlib.colors import LogNorm, TwoSlopeNorm

# UCI


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(224)
dataset="yacht"
x, y, xt, yt = prep_uci_data(f'{dataset}', device)
sum_sdmse = []
sum_mse = []
sum_cov = []

inds = []
with torch.no_grad():
    for i in range(3):
        with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/conformal-supp/uci/local/{dataset}/run-{i}/first/249999_parallel_model.p', 'rb') as f:

            ucimod = (pickle.load(f))

        cal_resids = (ucimod(x[1::2, :])['mean'] - y[1::2].unsqueeze(1)).view(y.shape[0] // 2, 101, 2).abs()

        quantiles = torch.quantile(cal_resids, 0.682, dim=0)


        min_index = torch.argmin(cal_resids.mean(0))

        # Convert the 1D index to 2D indices
        min_index_2d = (min_index // cal_resids.mean(0).size(1), min_index % cal_resids.mean(0).size(1))


        ind0 = min_index // cal_resids.mean(0).size(1)
        ind1 = min_index % cal_resids.mean(0).size(1)

        inds.append((ind0.item(), ind1))

        test_resids = (ucimod(xt)['mean'] - yt.unsqueeze(1)).view(yt.shape[0], 101, 2).abs()

        sum_cov.append(((test_resids[:, ind0, ind1] < quantiles[ind0, ind1]) * 1).float().mean())

        sum_sdmse.append((test_resids[:, ind0, ind1] - quantiles[ind0, ind1].unsqueeze(0)).pow(2).mean(0))
        sum_mse.append(test_resids.mean(0)[ind0, ind1])
    
sdmse_tensor = torch.tensor(sum_sdmse)
mse_tensor = torch.tensor(sum_mse)
cov_tensor = torch.tensor(sum_cov)

print(dataset)
print('sdmse')
print(sdmse_tensor.mean())
print(sdmse_tensor.var().pow(.5))

print('mse')
print(mse_tensor.mean())
print(mse_tensor.var().pow(.5))

print('mean cov')
print(cov_tensor.mean())
print(cov_tensor.var().pow(.5))

print(inds)


yacht
sdmse
tensor(0.0097)
tensor(0.)
mse
tensor(0.0477)
tensor(4.5625e-09)
mean cov
tensor(0.6176)
tensor(0.)
[(94, tensor(1, device='cuda:0')), (94, tensor(1, device='cuda:0')), (94, tensor(1, device='cuda:0'))]


/tmp/ipykernel_3164635/1863760073.py:25: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  min_index_2d = (min_index // cal_resids.mean(0).size(1), min_index % cal_resids.mean(0).size(1))
/tmp/ipykernel_3164635/1863760073.py:28: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ind0 = min_index // cal_resids.mean(0).size(1)


In [13]:
# hetero coverage

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(224)
dataset="power"
x, y, xt, yt = prep_uci_data(f'{dataset}', device)
sum_sdmse1 = []
sum_mse = []
sum_cov = []

inds = []
with torch.no_grad():
    for i in range(6):
        with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/conformal-supp/uci/local/{dataset}/run-{i}/first/49999_parallel_model.p', 'rb') as f:

            ucimod = (pickle.load(f))

        cal_resids1 = (ucimod(x[1::2, :])['mean'] - y[1::2].unsqueeze(1)).view(y.shape[0] // 2, 101, 2).abs()

        sds1 = ucimod(x[1::2, :])['precision'].pow(-.5).view(y.shape[0] // 2, 101, 2)
        conform_scores = cal_resids1 / sds1

        sdmses = (cal_resids1 - sds1).pow(2).mean(0)

        quantiles1 = torch.quantile(conform_scores, 0.682, dim=0)


        mn_min_index = torch.argmin(cal_resids1.mean(0), dim=0)
        sd_min_index = torch.argmin(sdmses, dim=0)


        ind0 = (mn_min_index[0] + sd_min_index[0]) // 2 
        ind1 = 0


        inds.append((ind0.item(), ind1))

        test_resids1 = (ucimod(xt)['mean'] - yt.unsqueeze(1)).view(yt.shape[0], 101, 2).abs()

        test_sds1 = ucimod(xt)['precision'].pow(-.5).view(yt.shape[0], 101, 2)

        sum_cov.append(((test_resids1[:, ind0, ind1] < quantiles1[ind0, ind1] * test_sds1[:, ind0, ind1]) * 1).float().mean())

        sum_sdmse1.append((test_resids1[:, ind0, ind1] - (quantiles1[ind0, ind1]  * test_sds1[:, ind0, ind1]).unsqueeze(0)).pow(2).mean(0).mean())
        sum_mse.append(test_resids1.mean(0)[ind0, ind1])
    
sdmse_tensor = torch.tensor(sum_sdmse1)
mse_tensor = torch.tensor(sum_mse)
cov_tensor = torch.tensor(sum_cov)

print('sdmse')
print(sdmse_tensor.mean())
print(sdmse_tensor.var().pow(.5))

print('mse')
print(mse_tensor.mean())
print(mse_tensor.var().pow(.5))

print('mean cov')
print(cov_tensor.mean())
print(cov_tensor.var().pow(.5))

print(inds)

/tmp/ipykernel_2828124/371586769.py:33: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ind0 = (mn_min_index[0] + sd_min_index[0]) // 2


sdmse
tensor(0.0305)
tensor(0.)
mse
tensor(0.1853)
tensor(1.6323e-08)
mean cov
tensor(0.6864)
tensor(0.)
[(92, 0), (92, 0), (92, 0), (92, 0), (92, 0), (92, 0)]


# Synthetic

In [63]:
datasetname = "3"
dataset = '3'

x, y, _ = prep_synth_data(dataset, device, seed=1000, sample_size=64, het=True)
xcal = x[1::2]
ycal = y[1::2]

xt, yt, _ = prep_synth_data(dataset, device, seed=1001, sample_size=64, het=True)
torch.manual_seed(99)
with torch.no_grad():
    for i in range(6):
        #with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/synth_exps/{datasetname}-conformal/run-{i}/first/19999_parallel_model.p', 'rb') as f:
        with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/conformal-supp/synth/local/{datasetname}/run-{i}/first/599999_parallel_model.p', 'rb') as f:
            mod1d = (pickle.load(f))

        cal_resids = (mod1d(xcal)['mean'] - ycal.unsqueeze(1)).abs()

        quantiles = torch.quantile(cal_resids, 0.682, dim=0)


        min_index = torch.argmin(cal_resids.mean(0))

        # Convert the 1D index to 2D indices
        min_index_2d = (min_index // cal_resids.mean(0).size(1), min_index % cal_resids.mean(0).size(1))


        ind0 = min_index // cal_resids.mean(0).size(1)
        ind1 = 0 #min_index % cal_resids.mean(0).size(1)

        test_resids = (mod1d(xt)['mean'] - yt.unsqueeze(1)).abs()

        sum_cov.append(((test_resids[:, ind0, ind1] < quantiles[ind0, ind1]) * 1).float().mean())

        sum_sdmse.append((test_resids[:, ind0, ind1] - quantiles[ind0, ind1].unsqueeze(0)).pow(2).mean(0))
        sum_mse.append(test_resids.mean(0)[ind0, ind1])
    
sdmse_tensor = torch.tensor(sum_sdmse)
mse_tensor = torch.tensor(sum_mse)
cov_tensor = torch.tensor(sum_cov)

print('sdmse')
print(sdmse_tensor.mean())
print(sdmse_tensor.var().pow(.5))

print('mse')
print(mse_tensor.mean())
print(mse_tensor.var().pow(.5))

print('mean cov')
print(cov_tensor.mean())
print(cov_tensor.var().pow(.5))

/tmp/ipykernel_3164635/3233387471.py:24: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  min_index_2d = (min_index // cal_resids.mean(0).size(1), min_index % cal_resids.mean(0).size(1))
/tmp/ipykernel_3164635/3233387471.py:27: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ind0 = min_index // cal_resids.mean(0).size(1)


sdmse
tensor(0.3482)
tensor(0.1379)
mse
tensor(0.5250)
tensor(0.1096)
mean cov
tensor(0.7135)
tensor(0.0468)


In [71]:
# hetero coverage
datasetname = "sine"
dataset = 'sine'

x, y, _ = prep_synth_data(dataset, device, seed=1000, sample_size=64, het=True)
xcal = x[1::2]
ycal = y[1::2]

xt, yt, _ = prep_synth_data(dataset, device, seed=1001, sample_size=64, het=True)
torch.manual_seed(99)

sum_sdmse1 = []
sum_mse = []
sum_cov = []

inds = []

with torch.no_grad():
    for i in range(6):
        #with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/synth_exps/{datasetname}-conformal/run-{i}/first/19999_parallel_model.p', 'rb') as f:
        with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/conformal-supp/synth/local/{datasetname}/run-{i}/first/599999_parallel_model.p', 'rb') as f:
            mod1d = (pickle.load(f))


        cal_resids1 = (mod1d(xcal)['mean'] - ycal.unsqueeze(1)).view(len(ycal), 101, 2).abs()

        sds1 = mod1d(xcal)['precision'].pow(.5).view(len(ycal), 101, 2)
        conform_scores = cal_resids1 / sds1

        sdmses = (cal_resids1 - sds1).pow(2).mean(0)

        quantiles1 = torch.quantile(conform_scores, 0.682, dim=0)


        mn_min_index = torch.argmin(cal_resids1.mean(0), dim=0)
        sd_min_index = torch.argmin(sdmses, dim=0)


        ind0 = (mn_min_index[0] + sd_min_index[0]) // 2 
        ind1 = 0


        inds.append((ind0.item(), ind1))

        test_resids1 = (mod1d(xt)['mean'] - yt.unsqueeze(1)).view(yt.shape[0], 101, 2).abs()

        test_sds1 = mod1d(xt)['precision'].pow(-.5).view(yt.shape[0], 101, 2)

        sum_cov.append(((test_resids1[:, ind0, ind1] < quantiles1[ind0, ind1] * test_sds1[:, ind0, ind1]) * 1).float().mean())

        sum_sdmse1.append((test_resids1[:, ind0, ind1] - (quantiles1[ind0, ind1]  * test_sds1[:, ind0, ind1]).unsqueeze(0)).pow(2).mean(0).mean())
        sum_mse.append(test_resids1.mean(0)[ind0, ind1])
    
sdmse_tensor = torch.tensor(sum_sdmse1)
mse_tensor = torch.tensor(sum_mse)
cov_tensor = torch.tensor(sum_cov)

print('sdmse')
print(sdmse_tensor.mean())
print(sdmse_tensor.var().pow(.5))

print('mse')
print(mse_tensor.mean())
print(mse_tensor.var().pow(.5))

print('mean cov')
print(cov_tensor.mean())
print(cov_tensor.var().pow(.5))

print(inds)

/tmp/ipykernel_3164635/330612738.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ind0 = (mn_min_index[0] + sd_min_index[0]) // 2


sdmse
tensor(0.3252)
tensor(7.6827e-06)
mse
tensor(0.8181)
tensor(1.5632e-06)
mean cov
tensor(0.6061)
tensor(6.5294e-08)
[(48, 0), (50, 0), (50, 0), (45, 0), (48, 0), (46, 0)]


In [56]:
cal_resids.view(len(y[1::2]), 101, 2).mean(0).argmin(dim=0)

tensor([93, 93], device='cuda:0')

In [21]:
np.random.seed(224)
dataset="power"
x, y, xt, yt = prep_uci_data(f'{dataset}', device)
i=0
with torch.no_grad():
    with open(f'./uci-exps/beta_nll-2/{dataset}/run-{i}/first/9999_parallel_model.p', 'rb') as f:

        ucibeta= (pickle.load(f))

    test_resids = (ucibeta(xt[:, :])['mean'] - yt[::].unsqueeze(1)).abs()

    mses = test_resids.pow(2).mean(0)

    sdmses = (test_resids - ucibeta(xt)['precision'].pow(-.5)).abs().pow(2).mean(0)
print('mse')
print(mses[0:6].mean())

print(mses[0:6].var().pow(.5))

print('sdmse')
print(sdmses[0:6].mean())
print(sdmses[0:6].var().pow(.5))

mse
tensor(0.1013, device='cuda:0')
tensor(0.0067, device='cuda:0')
sdmse
tensor(0.3081, device='cuda:0')
tensor(0.3682, device='cuda:0')


In [27]:
np.random.seed(224)
dataset="power"
x, y, xt, yt = prep_uci_data(f'{dataset}', device)
i=0
with open(f'./uci-exps/beta_nll-2/{dataset}/run-{i}/first/4999_parallel_model.p', 'rb') as f:
    
    ucibeta= (pickle.load(f))

test_resids = (ucibeta(xt[:, :])['mean'] - yt[::].unsqueeze(1)).abs()

mses = test_resids.pow(2).mean(0)

sdmses = (test_resids - ucibeta(xt)['precision'].pow(-.5)).abs().pow(2).mean(0)
print('mse')
print(mses[0:6].mean())

print(mses[0:6].var().pow(.5))

print('sdmse')
print(sdmses[0:6].mean())
print(sdmses[0:6].var().pow(.5))

mse
tensor(0.0943, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0058, device='cuda:0', grad_fn=<PowBackward0>)
sdmse
tensor(1.0137, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0066, device='cuda:0', grad_fn=<PowBackward0>)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(224)
datasetname = "sine"
dataset = 'sine'

sum_sdmse = []
sum_mse = []

x, y, _ = prep_synth_data(dataset, device, seed=1000, sample_size=64, het=True)


xt, yt, _ = prep_synth_data(dataset, device, seed=1001, sample_size=64, het=True)
with open(f'./uci-exps/beta_nll-2/{dataset}/run-{i}/first/4999_parallel_model.p', 'rb') as f:
    
    ucibeta= (pickle.load(f))

test_resids = (ucibeta(xt[:, :])['mean'] - yt[::].unsqueeze(1)).abs()

mses = test_resids.pow(2).mean(0)

sdmses = (test_resids - ucibeta(xt)['precision'].pow(-.5)).abs().pow(2).mean(0)
print('mse')
print(mses[0:6].mean())

print(mses[0:6].var().pow(.5))

print('sdmse')
print(sdmses[0:6].mean())
print(sdmses[0:6].var().pow(.5))

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(224)
datasetname = "sine"
dataset = 'sine'

sum_sdmse = []
sum_mse = []

x, y, _ = prep_synth_data(dataset, device, seed=1000, sample_size=64, het=True)


xt, yt, _ = prep_synth_data(dataset, device, seed=1001, sample_size=64, het=True)

for i in range(3):
    with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/synth_exps/rebuttal/{datasetname}/run-{i}/first/19999_parallel_model.p', 'rb') as f:
    
        modl2 = (pickle.load(f))



    test_resids = (modl2(xt)['mean'] - yt.unsqueeze(1)).abs()
    
    mse = test_resids.pow(2).mean(0).view(101, 2)

    sdmse = (test_resids - modl2(xt)['precision'].pow(-.5)).pow(2).mean(0).view(101, 2)
    

    
    sum_sdmse.append(sdmse)
    sum_mse.append(mse)
    
sdmse_tensor = torch.cat(sum_sdmse, dim=1)
mse_tensor = torch.cat(sum_mse, dim=1)


print('sdmse')
print(sdmse_tensor.mean())
print(sdmse_tensor.var().pow(.5))

print('mse')
print(mse_tensor.mean())
print(mse_tensor.var().pow(.5))


sdmse
tensor(0.4933, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0909, device='cuda:0', grad_fn=<PowBackward0>)
mse
tensor(0.9653, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0619, device='cuda:0', grad_fn=<PowBackward0>)


In [9]:




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(224)
datasetname = "sine"
dataset = 'sine'

sum_sdmse = []
sum_mse = []

x, y, _ = prep_synth_data(dataset, device, seed=1000, sample_size=64, het=True)


xt, yt, _ = prep_synth_data(dataset, device, seed=1001, sample_size=64, het=True)
with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/synth_exps/rebuttal/sine-beta-nll/run-0/first/49999_parallel_model.p', 'rb') as f:
    
    beta1d= (pickle.load(f))

test_resids = (beta1d(xt[:, :])['mean'] - yt[::].unsqueeze(1)).abs()

mses = test_resids.pow(2).mean(0)

sdmses = (test_resids - beta1d(xt)['precision'].pow(-.5)).abs().pow(2).mean(0)
print('mse')
print(mses[0:6].mean())

print(mses[0:6].var().pow(.5))

print('sdmse')
print(sdmses[0:6].mean())
print(sdmses[0:6].var().pow(.5))

mse
tensor(0.6898, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0529, device='cuda:0', grad_fn=<PowBackward0>)
sdmse
tensor(0.5272, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0668, device='cuda:0', grad_fn=<PowBackward0>)


In [3]:
sdmse_tensor.min(dim=0)

torch.return_types.min(
values=tensor([0.4624, 0.4625, 0.4627, 0.4625, 0.4627, 0.4625], device='cuda:0',
       grad_fn=<MinBackward0>),
indices=tensor([70, 86, 15, 62, 27, 34], device='cuda:0'))

In [4]:
min_inds = (mse_tensor.argmin(dim=0) + sdmse_tensor.argmin(dim=0)) // 2

print(min_inds)

tensor([82, 88, 53, 78, 59, 62], device='cuda:0')


/tmp/ipykernel_2262608/2493637657.py:1: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  min_inds = (mse_tensor.argmin(dim=0) + sdmse_tensor.argmin(dim=0)) // 2


In [8]:
(sdmse_tensor.argmin(dim=0) + mse_tensor.argmin(dim=0)) // 2

/tmp/ipykernel_2259298/4141731975.py:1: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  (sdmse_tensor.argmin(dim=0) + mse_tensor.argmin(dim=0)) // 2


tensor([82, 88, 53, 78, 59, 62], device='cuda:0')

In [2]:
np.random.seed(224)
dataset="climate"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_test=torch.tensor(np.load('/home/ewongtoi/Documents/heteroskedastic_nns/climate/val_input.npy')).to(device)
y_test_all=torch.tensor(np.load('/home/ewongtoi/Documents/heteroskedastic_nns/climate/val_target.npy')).to(device)

x=torch.tensor(np.load('/home/ewongtoi/Documents/heteroskedastic_nns/climate/train_input.npy')).to(device)
y=torch.tensor(np.load('/home/ewongtoi/Documents/heteroskedastic_nns/climate/train_target.npy')).to(device)


xcal = x[1::2400, :]
ycal = y[1::2400, 123]

xtestsub = x_test[::1000, :]
ytest = y_test_all[::1000, 123]



del(x)
del(y)

del(x_test)
del(y_test_all)

In [35]:
with torch.no_grad():
    with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/climate_exps/solar-flux-beta-nll/run-0/first/11_parallel_model.p', 'rb') as f:

        climbeta= (pickle.load(f))


    test_outs = climbeta(x_test[::10])
    test_resids = (test_outs['mean'] - y_test_all[::10, 124].unsqueeze(1).unsqueeze(1)).abs()

    mses = test_resids.pow(2).mean(0)

    sdmses = (test_resids - test_outs['precision'].pow(-.5)).abs().pow(2).mean(0)
print('mse')
print(mses[0:6].mean())

print(mses[0:6].var().pow(.5))

print('sdmse')
print(sdmses[0:6].mean())
print(sdmses[0:6].var().pow(.5))

mse
tensor(0.3771, device='cuda:0')
tensor(7.5314e-05, device='cuda:0')
sdmse
tensor(0.3217, device='cuda:0')
tensor(0.0001, device='cuda:0')


In [3]:
# hetero coverage

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(224)

sum_sdmse1 = []
sum_mse = []
sum_cov = []

inds = []
with torch.no_grad():
    for i in range(6):
        with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/conformal-supp/climate/solar-flux-local/run-{i}/first/24_parallel_model.p', 'rb') as f:

            climmod = (pickle.load(f))

        cal_resids1 = (climmod(xcal)['mean'] - ycal.unsqueeze(1).unsqueeze(1)).view(ycal.shape[0], 101, 2).abs()

        sds1 = climmod(xcal)['precision'].pow(-.5).view(ycal.shape[0], 101, 2)
        conform_scores = cal_resids1 / sds1

        sdmses = (cal_resids1 - sds1).pow(2).mean(0)

        quantiles1 = torch.quantile(conform_scores, 0.682, dim=0)


        mn_min_index = torch.argmin(cal_resids1.mean(0), dim=0)
        sd_min_index = torch.argmin(sdmses, dim=0)


        ind0 = (mn_min_index[0] + sd_min_index[0]) // 2 
        ind1 = 0


        inds.append((ind0.item(), ind1))

        test_resids1 = (climmod(xtestsub)['mean'] - ytest.unsqueeze(1).unsqueeze(1)).view(ytest.shape[0], 101, 2).abs()

        test_sds1 = climmod(xtestsub)['precision'].pow(-.5).view(len(ytest), 101, 2)

        sum_cov.append(((test_resids1[:, ind0, ind1] < quantiles1[ind0, ind1] * test_sds1[:, ind0, ind1]) * 1).float().mean())

        sum_sdmse1.append((test_resids1[:, ind0, ind1] - (quantiles1[ind0, ind1]  * test_sds1[:, ind0, ind1]).unsqueeze(0)).pow(2).mean(0).mean())
        sum_mse.append(test_resids1.mean(0)[ind0, ind1])
    
sdmse_tensor = torch.tensor(sum_sdmse1)
mse_tensor = torch.tensor(sum_mse)
cov_tensor = torch.tensor(sum_cov)

print('sdmse')
print(sdmse_tensor.mean())
print(sdmse_tensor.var().pow(.5))

print('mse')
print(mse_tensor.mean())
print(mse_tensor.var().pow(.5))

print('mean cov')
print(cov_tensor.mean())
print(cov_tensor.var().pow(.5))

print(inds)

/tmp/ipykernel_2884246/2521851906.py:32: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ind0 = (mn_min_index[0] + sd_min_index[0]) // 2


sdmse
tensor(0.0111)
tensor(1.1017e-05)
mse
tensor(0.0486)
tensor(9.1057e-05)
mean cov
tensor(0.5666)
tensor(0.0196)
[(20, 0), (19, 0), (8, 0), (7, 0), (7, 0), (23, 0)]


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(224)
dataset="climate"

sum_sdmse = []
sum_mse = []
sum_cov = []

inds = []
with torch.no_grad():
    for i in range(6):
        with open(f'/extra/ucibdl0/ewongtoi/heteroskedastic_nns/conformal-supp/climate/solar-flux-local/run-{i}/first/24_parallel_model.p', 'rb') as f:

            climmod = (pickle.load(f))

        cal_resids = (climmod(xcal)['mean'] - ycal.unsqueeze(1).unsqueeze(1)).view(ycal.shape[0], 101, 2).abs()

        quantiles = torch.quantile(cal_resids, 0.682, dim=0)


        min_index = torch.argmin(cal_resids.mean(0))

        # Convert the 1D index to 2D indices
        min_index_2d = (min_index // cal_resids.mean(0).size(1), min_index % cal_resids.mean(0).size(1))


        ind0 = min_index // cal_resids.mean(0).size(1)
        ind1 = min_index % cal_resids.mean(0).size(1)

        inds.append((ind0.item(), ind1))

        test_resids = (climmod(xtestsub)['mean'] - ytest.unsqueeze(1).unsqueeze(1)).view(len(ytest), 101, 2).abs()

        sum_cov.append(((test_resids[:, ind0, ind1] < quantiles[ind0, ind1]) * 1).float().mean())

        sum_sdmse.append((test_resids[:, ind0, ind1] - quantiles[ind0, ind1].unsqueeze(0)).pow(2).mean(0))
        sum_mse.append(test_resids.mean(0)[ind0, ind1])
    
sdmse_tensor = torch.tensor(sum_sdmse)
mse_tensor = torch.tensor(sum_mse)
cov_tensor = torch.tensor(sum_cov)

print(dataset)
print('sdmse')
print(sdmse_tensor.mean())
print(sdmse_tensor.var().pow(.5))

print('mse')
print(mse_tensor.mean())
print(mse_tensor.var().pow(.5))

print('mean cov')
print(cov_tensor.mean())
print(cov_tensor.var().pow(.5))

print(inds)

/tmp/ipykernel_3164635/2356358947.py:25: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  min_index_2d = (min_index // cal_resids.mean(0).size(1), min_index % cal_resids.mean(0).size(1))
/tmp/ipykernel_3164635/2356358947.py:28: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ind0 = min_index // cal_resids.mean(0).size(1)


climate
sdmse
tensor(0.0111)
tensor(3.4375e-06)
mse
tensor(0.0485)
tensor(4.4173e-06)
mean cov
tensor(0.5943)
tensor(0.0153)
[(12, tensor(1, device='cuda:0')), (30, tensor(1, device='cuda:0')), (12, tensor(1, device='cuda:0')), (10, tensor(1, device='cuda:0')), (92, tensor(1, device='cuda:0')), (42, tensor(0, device='cuda:0'))]
